**Objectifs**

*   Faire un graphique du top 10 des pays les plus grand
*   Puis un top 10 des pays avec le plus d'habitant au mètre carré


Lien du pitch : https://drive.google.com/file/d/1PSowEJ16Xf5rqmALuhmTi16U4MEucLk7/view

**Import**

In [0]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as ofli
from IPython.display import display
import ipywidgets as widgets

**Authentification**

**Source : Kaggle =>** https://www.kaggle.com/sudalairajkumar/undata-country-profiles/data#country_profile_variables.csv

From SRK, last updated in 2018

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1f-yl7fIkJYyTwJRRqeza5kWompmClYNE"})
downloaded.GetContentFile('country_profile_variables.csv')

In [0]:
data = pd.read_csv("country_profile_variables.csv")
data

,country,Region,Surface area (km2),Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),Economy: Industry (% of GVA),Economy: Services and other activity (% of GVA),Employment: Agriculture (% of employed),Employment: Industry (% of employed),Employment: Services (% of employed),Unemployment (% of labour force),Labour force participation (female/male pop. %),Agricultural production index (2004-2006=100),Food production index (2004-2006=100),International trade: Exports (million US$),International trade: Imports (million US$),International trade: Balance (million US$),"Balance of payments, current account (million US$)",Population growth rate (average annual %),Urban population (% of total population),Urban population growth rate (average annual %),"Fertility rate, total (live births per woman)","Life expectancy at birth (females/males, years)","Population age distribution (0-14 / 60+ years, %)",International migrant stock (000/% of total pop.),Refugees and others of concern to UNHCR (in thousands),Infant mortality rate (per 1000 live births,Health: Total expenditure (% of GDP),Health: Physicians (per 1000 pop.),Education: Government expenditure (% of GDP),Education: Primary gross enrol. ratio (f/m per 100 pop.),Education: Secondary gross enrol. ratio (f/m per 100 pop.),Education: Tertiary gross enrol. ratio (f/m per 100 pop.),Seats held by women in national parliaments %,Mobile-cellular subscriptions (per 100 inhabitants),Mobile-cellular subscriptions (per 100 inhabitants).1,Individuals using the Internet (per 100 inhabitants),Threatened species (number),Forested area (% of land area),CO2 emission estimates (million tons/tons per capita),"Energy production, primary (Petajoules)",Energy supply per capita (Gigajoules),"Pop. using improved drinking water (urban/rural, %)","Pop. using improved sanitation facilities (urban/rural, %)",Net Official Development Assist. received (% of GNI)
0,Afghanistan,SouthernAsia,652864,35530,54.4,106.3,20270,-2.4,623.2,23.3,23.3,53.3,61.6,10.0,28.5,8.6,19.3/83.6,125,125,1458,3568,-2110,-5121,3.2,26.7,4.0,5.3,63.5/61.0,43.2/4.1,382.4/1.2,1513.1,68.6,8.2,0.3,3.3,91.1/131.6,39.7/70.7,3.7/13.3,27.7,61.6,8.3,42,2.1,9.8/0.3,63,5,78.2/47.0,45.1/27.0,21.43,-99
1,Albania,SouthernEurope,28748,2930,106.9,101.9,11541,2.6,3984.2,22.4,26.0,51.7,41.4,18.3,40.3,15.8,40.2/61.0,134,134,1962,4669,-2707,-1222,-0.1,57.4,2.2,1.7,79.9/75.6,17.4/19.0,57.6/2.0,8.8,14.6,5.9,1.3,3.5,111.7/115.5,92.5/98.8,68.1/48.7,22.9,106.4,63.3,130,28.2,5.7/2.0,84,36,94.9/95.2,95.5/90.2,2.96,-99
2,Algeria,NorthernAfrica,2381741,41318,17.3,102.0,164779,3.8,4154.1,12.2,37.3,50.5,10.8,34.5,54.7,11.4,17.0/70.7,160,161,29992,47091,-17099,-27229,2.0,70.7,2.8,3.0,76.5/74.1,29.3/9.4,242.4/0.6,99.8,27.7,7.2,...,...,112.7/119.5,101.7/98.1,45.1/28.9,31.6,113.0,38.2,135,0.8,145.4/3.7,5900,55,84.3/81.8,89.8/82.2,0.05,-99
3,American Samoa,Polynesia,199,56,278.2,103.6,-99,-99,-99.0,-99,-99.0,-99.0,...,...,...,...,.../...,112,112,-99,-99,-99,-99,-~0.0,87.2,-0.1,2.6,77.8/71.1,33.3/9.0,23.2/41.8,-99,9.6,-99.0,-99,-99,-99,-99,-99,-99.0,...,-99,92,87.9,-99,-99,-99,100.0/100.0,62.5/62.5,-99,-99
4,Andorra,SouthernEurope,468,77,163.8,102.3,2812,0.8,39896.4,0.5,10.8,88.6,-99,-99,-99,-99,-99,-99,-99,100,1355,-1255,-99,-1.6,85.1,0.1,1.2,-99,14.4/19.0,42.1/59.7,-99,-99,8.1,3.7,3.3,-99,-99,-99,32.1,88.1,96.9,13,34.0,0.5/6.4,1,119,100.0/100.0,100.0/100.0,-99,-99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Wallis and Futuna Islands,Polynesia,142,12,84.1,93.4,-99,-99,-99.0,-99,-99.0,-99.0,-99,-99,-99,-99,-99,115,115,1,53,-51,-99,-2.1,0.0,0.0,2.1,78.7/72.8,25.5/15.4,2.8/21.7,-99,...,-99.0,-99,-99,-99,-99,-99

**Filtrage des données**

In [0]:
data = pd.read_csv("country_profile_variables.csv", usecols=["country", "Surface area (km2)", "Population density (per km2, 2017)"])
# for index, row in data.iterrows():
#   print(row["Surface area (km2)"])

# Suppression de la ligne Holy see inutile
data = data.loc[data['country'] != 'Holy See']

# Tri de la surface des pays
data['Surface area (km2)'] = pd.to_numeric(data['Surface area (km2)'])
data['Population density (per km2, 2017)'] = pd.to_numeric(data['Population density (per km2, 2017)'])
data.rename(columns = {'Surface area (km2)':'SURFACE', 'Population density (per km2, 2017)':'DENSITY', 
                              'country':'COUNTRY'}, inplace = True) 

# Suppression de tout sauf les 10 pays ayant le plus de surface
data

,COUNTRY,SURFACE,DENSITY
0,Afghanistan,652864,54.4
1,Albania,28748,106.9
2,Algeria,2381741,17.3
3,American Samoa,199,278.2
4,Andorra,468,163.8
...,...,...,...
224,Wallis and Futuna Islands,142,84.1
225,Western Sahara,266000,2.1
226,Yemen,527968,53.5
227,Zambia,752612,23.0


**Affichage graphique** : Séparation de data en deux tableaux (un densité et un surface)

In [0]:
#Tri de deux tableaux (surface / densite au km²)
limit = 10
ordered_densite = data
ordered_densite = ordered_densite.sort_values(by="DENSITY",ascending=True)
ordered_densite = ordered_densite.tail(limit)
ordered_surface = data
ordered_surface = ordered_surface.sort_values(by="SURFACE",ascending=True)
ordered_surface = ordered_surface.tail(limit)
ordered_surface

,COUNTRY,SURFACE,DENSITY
2,Algeria,2381741,17.3
106,Kazakhstan,2724902,6.7
8,Argentina,2780400,16.2
95,India,3287263,450.4
11,Australia,7692060,3.2
28,Brazil,8515767,25.0
45,China,9600000,150.1
217,United States of America,9833517,35.5
37,Canada,9984670,4.0
167,Russian Federation,17098246,8.8


**Affichage graphique :** On crée 2 nouveaux tableaux pour les titres qui seront affichés sur les barres

In [0]:
ordered_surface_titles = []
ordered_densite_titles = []
# Afin d'afficher un joli titre, on est recrée un tableau de chaines de caractères
# dans avec les nombres avec des espaces + km²
for surface in ordered_surface.iterrows():
  adding_commas=format(surface[1][1], "8,d")
  replacing_with_spaces=adding_commas.replace(",", " ")
  ordered_surface_titles.append(replacing_with_spaces+" km²")

# On fait pareil pour la densité
for density in ordered_densite.iterrows():
  adding_commas=format(int(density[1][2]), "8,d")
  replacing_with_spaces=adding_commas.replace(",", " ")
  ordered_densite_titles.append(replacing_with_spaces+" h/km²")

bars=[
  go.Bar(text=ordered_surface_titles,
         textposition='auto',
         name="Surface", 
         showlegend=True,
         y=ordered_surface["COUNTRY"]+"  ", 
         x=ordered_surface["SURFACE"],
         orientation='h', 
         marker_color='rgb(25, 84, 5)',
         marker_line_color='rgb(50, 84, 50)'
        ),
  go.Bar(text=ordered_densite_titles,
            textposition='auto',
         name="Densité", 
         showlegend=True,
         y=ordered_densite["COUNTRY"]+"  ",
         x=ordered_densite["DENSITY"],
         orientation='h',
         visible=False, 
         marker_color='rgb(255, 195, 118)',
         marker_line_color='rgb(170, 68, 1)'
         ),
]
layout = go.Layout(
    plot_bgcolor='rgb(255, 255, 255)',
  title='Top '+str(limit)+' biggest Countries',
  updatemenus=list([
    dict(
         showactive=True, 
         direction = "left",
         type="buttons", 
         active=0, 
         buttons=[
          {'label': 'Surface', 'method': 'update', 'args': [{'visible': [True, False]}]},
          {'label': 'Densite', 'method': 'update', 'args': [{'visible': [False, True]}]},
        ],
        pad={"r": 10, "t": 10},
        x=0.987,
        xanchor="left",
        y=1.2,
        yanchor="top"
    )
  ]),
  annotations=[dict(
    text = 'Updated in 2018 from The World Bank',
    showarrow = False,
    xref = 'paper', x = 0.95,
    yref = 'paper', y = -0.15),]
)


fig =go.Figure(bars, layout)
fig.update_layout(height=limit*60)
fig.show()



**Bouton de téléchargement**

In [0]:
def clicked(arg):
  ofli.iplot(fig,image='png',image_width=1920, image_height=limit*60, filename='Top '+str(limit)+' biggest Countries')

button_download = widgets.Button(description = 'Export Graph as png')
button_download.on_click(clicked)
display(button_download)

Button(description='Export Graph as png', style=ButtonStyle())